1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM 질문과 같이 전달

## 1. 패키지 설치

In [ ]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone

## 2. Knowledge Base 구성을 위한 데이터 생성

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# loader = Docx2txtLoader('./tax_with_table.docx')
loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
document_list[52]

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


# database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [ ]:
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [ ]:
from tqdm.auto import tqdm
import time

# 청크 크기 기반으로 배치 크기 계산
avg_chunk_size = 1500  # 각 청크의 문자 수
embedding_dim = 768    # ko-sbert-sts의 임베딩 차원
bytes_per_float = 4    # 각 임베딩 값당 바이트 수
safety_factor = 0.8    # 안전 마진

# 배치당 최대 문서 수 계산 (최소값 설정)
# (2MB 제한) / (청크당 예상 크기) * 안전 계수
max_batch_size = max(
    1,  # 최소 배치 크기
    int((2 * 1024 * 1024) / (avg_chunk_size * embedding_dim * bytes_per_float) * safety_factor)
)

print(f"Calculated optimal batch size: {max_batch_size}")

# 배치 처리로 문서 업로드
# 작은 배치 크기로 시작
initial_batch_size = min(20, max_batch_size)  # 안전한 초기값

for i in tqdm(range(0, len(document_list), initial_batch_size), desc="Uploading documents"):
    batch = document_list[i:i + initial_batch_size]

    try:
        database.add_documents(documents=batch)
        time.sleep(0.5)

    except Exception as e:
        print(f"Error in batch {i//initial_batch_size}: {str(e)}")
        print(f"Reducing batch size and retrying...")

        # 오류 발생 시 배치 크기 절반으로 줄여서 재시도
        smaller_batch = batch[:max(1, len(batch)//2)]
        try:
            database.add_documents(documents=smaller_batch)
            time.sleep(0.5)
        except Exception as retry_error:
            print(f"Retry failed: {str(retry_error)}")

print(f"Total documents processed: {len(document_list)}")

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

## 3. 답변 생성을 위한 Retrieval
  - RetrievalQA에 전달하기 위해 retriever 생성
  - search_kwargs 의 k 값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
  - .invoke() 를 호출해서 어떤 문서를 가져오는지 확인 가능

In [ ]:
# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retriever = database.as_retriever(search_kwargs={'k': 7})
retriever.invoke(query)

## 4. Augmentation을 위한 Prompt 활용
  - Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
ai_message = qa_chain.invoke({"query": query})

In [ ]:
ai_message

## 6. Retrieval을 위한 keyword 사전 활용
  - Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정
  - LangChain Expression Language (LCEL)을 활용한 Chain 연계

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}
    사용자의 질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [ ]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [ ]:
ai_response = tax_chain.invoke({"question": query})

In [ ]:
ai_response